In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

%matplotlib inline

In [ ]:
version = 8

In [ ]:
records_iq = pd.read_csv(
    './generated/1-features-iq.csv', 
    parse_dates=['week_start_date'], 
    index_col='week_start_date').interpolate()
records_sj = pd.read_csv(
    './generated/1-features-sj.csv', 
    parse_dates=['week_start_date'], 
    index_col='week_start_date').interpolate()

In [ ]:
training_targets_iq = pd.read_csv(
    './generated/1-labels-train-iq.csv', 
    parse_dates=['week_start_date'], 
    index_col='week_start_date')
training_targets_sj = pd.read_csv(
    './generated/1-labels-train-sj.csv', 
    parse_dates=['week_start_date'], 
    index_col='week_start_date')

In [ ]:
stl_dec_iq = pd.read_csv(
    './generated/2-time-dec-features-iq.csv', 
    parse_dates=['week_start_date'], 
    index_col='week_start_date')
stl_dec_sj = pd.read_csv(
    './generated/2-time-dec-features-sj.csv', 
    parse_dates=['week_start_date'], 
    index_col='week_start_date')

In [ ]:
records_train_iq = pd.concat([records_iq.head(520), training_targets_iq[['total_cases']]], axis=1)
records_train_sj = pd.concat([records_sj.head(936), training_targets_sj[['total_cases']]], axis=1)

stl_dec_train_iq = pd.concat([stl_dec_iq.head(520), training_targets_iq[['total_cases']]], axis=1)
stl_dec_train_sj = pd.concat([stl_dec_sj.head(936), training_targets_sj[['total_cases']]], axis=1)
stl_dec_test_iq = stl_dec_iq.tail(156)
stl_dec_test_sj = stl_dec_sj.tail(260)

In [ ]:
def lagged_corr(df, skip_col, label_col, lag):
    lcorr = pd.concat([
        df.drop(skip_col, axis=1).shift(lag),
        df[[label_col]]], axis=1).corr()

    lcorr_label = lcorr[label_col].drop(label_col)

    return lcorr_label

In [ ]:
records_multi_lagged_corr_iq = pd.DataFrame(
    {i: lagged_corr(records_train_iq, ['total_cases', 'weekofyear'], 'total_cases', i) for i in range(25)}
).T

In [ ]:
plt.figure(figsize=(16, 8))
sns.heatmap(records_multi_lagged_corr_iq, robust=True, center=0)
plt.title('Iquitos feature/label correlations')

In [ ]:
records_multi_lagged_corr_sj = pd.DataFrame(
    {i: lagged_corr(records_train_sj, ['total_cases', 'weekofyear'], 'total_cases', i) for i in range(25)}
).T

In [ ]:
plt.figure(figsize=(16, 8))
sns.heatmap(records_multi_lagged_corr_sj, robust=True, center=0)
plt.title('San Juan feature/label correlations')

In [ ]:
stl_dec_multi_lagged_corr_iq = pd.DataFrame(
    {i: lagged_corr(stl_dec_train_iq, ['total_cases'], 'total_cases', i) for i in range(25)}
).T

In [ ]:
plt.figure(figsize=(16, 8))
sns.heatmap(stl_dec_multi_lagged_corr_iq, robust=True, center=0)
plt.title('Iquitos STLdec/label correlations')

In [ ]:
stl_dec_multi_lagged_corr_sj = pd.DataFrame(
    {i: lagged_corr(stl_dec_train_sj, ['total_cases'], 'total_cases', i) for i in range(25)}
).T

In [ ]:
plt.figure(figsize=(16, 8))
sns.heatmap(stl_dec_multi_lagged_corr_sj, robust=True, center=0)
plt.title('San Juan STLdec/label correlations')

In [ ]:
records_ml_corr_max_iq = pd.concat({
    'max_iq': records_multi_lagged_corr_iq.max(),
    'min_iq': records_multi_lagged_corr_iq.min(),
}, axis=1)

records_ml_corr_argmax_iq = pd.concat({
    'argmax_iq': records_multi_lagged_corr_iq.idxmax(), 
    'argmin_iq': records_multi_lagged_corr_iq.idxmin(), 
}, axis=1)

records_ml_corr_max_sj = pd.concat({
    'max_sj':records_multi_lagged_corr_sj.max(),
    'min_sj': records_multi_lagged_corr_sj.min(),
}, axis=1)

records_ml_corr_argmax_sj = pd.concat({
    'argmax_sj': records_multi_lagged_corr_sj.idxmax(),  
    'argmin_sj': records_multi_lagged_corr_sj.idxmin(), 
}, axis=1)

pd.concat([records_ml_corr_argmax_iq, records_ml_corr_argmax_sj, records_ml_corr_max_iq, 
           records_ml_corr_max_sj], axis=1, sort=True)

In [ ]:
stl_dec_ml_corr_max_iq = pd.concat({
    'max_iq': stl_dec_multi_lagged_corr_iq.max(),
    'min_iq': stl_dec_multi_lagged_corr_iq.min(),
}, axis=1, sort=True)

stl_dec_ml_corr_argmax_iq = pd.concat({
    'argmax_iq': stl_dec_multi_lagged_corr_iq.idxmax(), 
    'argmin_iq': stl_dec_multi_lagged_corr_iq.idxmin(), 
}, axis=1, sort=True)

stl_dec_ml_corr_max_sj = pd.concat({
    'max_sj': stl_dec_multi_lagged_corr_sj.max(),
    'min_sj': stl_dec_multi_lagged_corr_sj.min()
}, axis=1, sort=True)

stl_dec_ml_corr_argmax_sj = pd.concat({
    'argmax_sj': stl_dec_multi_lagged_corr_sj.idxmax(), 
    'argmin_sj': stl_dec_multi_lagged_corr_sj.idxmin(), 
}, axis=1, sort=True)


pd.concat([stl_dec_ml_corr_argmax_iq, stl_dec_ml_corr_argmax_iq, stl_dec_ml_corr_max_sj, 
           stl_dec_ml_corr_max_sj], axis=1, sort=True)

In [ ]:
plt.figure(figsize=(16, 4))
sns.heatmap(records_ml_corr_max_iq.T, 
            cmap=sns.color_palette('BrBG', n_colors=5), 
            robust=True, 
            square=True, 
            center=0,
            annot=records_ml_corr_argmax_iq.T)
plt.title('Maximum lagged cross correlation iq')

In [ ]:
plt.figure(figsize=(16, 4))
sns.heatmap(records_ml_corr_max_sj.T, 
            cmap=sns.color_palette('BrBG', n_colors=5), 
            robust=True, 
            square=True, 
            center=0,
            annot=records_ml_corr_argmax_sj.T)
plt.title('Maximum lagged cross correlation sj')

In [ ]:
plt.figure(figsize=(16, 4))
stl_dec_ml_corr_max_filter_iq = list(map(
    lambda s: s.startswith('seasonal') or s.startswith('trend'), 
    stl_dec_ml_corr_max_iq.index.values))
sns.heatmap(
    stl_dec_ml_corr_max_iq[stl_dec_ml_corr_max_filter_iq].T, 
    cmap=sns.color_palette('BrBG', n_colors=5), 
    robust=True, 
    square=True, 
    center=0,
    annot=stl_dec_ml_corr_argmax_iq[stl_dec_ml_corr_max_filter_iq].T)
plt.title('Maximum lagged cross correlation for STL decomposed seasonal data iq')

In [ ]:
plt.figure(figsize=(16, 4))
stl_dec_ml_corr_max_filter_sj = list(map(
    lambda s: s.startswith('seasonal') or s.startswith('trend'), 
    stl_dec_ml_corr_max_sj.index.values))
sns.heatmap(
    stl_dec_ml_corr_max_sj[stl_dec_ml_corr_max_filter_sj].T, 
    cmap=sns.color_palette('BrBG', n_colors=5), 
    robust=True, 
    square=True, 
    center=0,
    annot=stl_dec_ml_corr_argmax_sj[stl_dec_ml_corr_max_filter_sj].T)
plt.title('Maximum lagged cross correlation for STL decomposed seasonal data sj')

In [ ]:
def feature_autoselect(corr_max, corr_argmax, city, ntake, max_lag):
    pointer_min = 0
    pointer_max = 0
    f_selected = []
    f_lags = []
    f_count = len(corr_max)
    corr_abs = corr_max.abs()
    fc_max = corr_abs['max_' + city].sort_values(ascending=False)
    fc_min = corr_abs['min_' + city].sort_values(ascending=False)
    ca_max = corr_argmax['argmax_' + city]
    ca_min = corr_argmax['argmin_' + city]
    
    while True:
        if len(f_selected) >= ntake: break
        if pointer_max >= f_count and pointer_min >= f_count: 
            break
        elif pointer_max >= f_count and pointer_min < f_count:
            if (fc_min.index.values[pointer_min] not in f_selected) \
            and (ca_min[fc_min.index[pointer_min]] <= max_lag):
                f_selected.append(fc_min.index.values[pointer_min])
                f_lags.append(ca_min[fc_min.index[pointer_min]])
            
            pointer_min += 1
        elif pointer_min >= f_count and pointer_max < f_count:
            if (fc_max.index.values[pointer_max] not in f_selected) \
            and (ca_max[fc_max.index[pointer_max]] <= max_lag):
                f_selected.append(fc_max.index.values[pointer_max])
                f_lags.append(ca_max[fc_max.index[pointer_max]])
                
            pointer_max += 1
        else:
            if fc_min[pointer_min] > fc_max[pointer_max]:
                if (fc_min.index.values[pointer_min] not in f_selected) \
                and (ca_min[fc_min.index[pointer_min]] <= max_lag):
                    f_selected.append(fc_min.index.values[pointer_min])
                    f_lags.append(ca_min[fc_min.index[pointer_min]])
                pointer_min += 1
            else:
                if (fc_max.index.values[pointer_max] not in f_selected) \
                and (ca_max[fc_max.index[pointer_max]] <= max_lag):
                    f_selected.append(fc_max.index.values[pointer_max])
                    f_lags.append(ca_max[fc_max.index[pointer_max]])
                pointer_max += 1
    
    return dict(zip(f_selected, f_lags))

In [ ]:
def select_lag(source_df, select_lag):
    stl_ref_iq = pd.DataFrame(index=source_df.index)
    for k, v in select_lag.items():
        stl_ref_iq[k] = source_df[k].shift(v)

    return stl_ref_iq

In [ ]:
selected_f_stl_iq = feature_autoselect(corr_max=stl_dec_ml_corr_max_iq, 
                                       corr_argmax=stl_dec_ml_corr_argmax_iq, city='iq', max_lag=18, ntake=15)
selected_f_stl_sj = feature_autoselect(corr_max=stl_dec_ml_corr_max_sj, 
                                       corr_argmax=stl_dec_ml_corr_argmax_sj, city='sj', max_lag=18, ntake=15)

In [ ]:
selected_f_stl_iq

In [ ]:
selected_f_stl_sj

In [ ]:
refined_stl_iq = select_lag(stl_dec_iq, selected_f_stl_iq)
refined_stl_sj = select_lag(stl_dec_sj, selected_f_stl_sj)

In [ ]:
features_1_train_iq = pd.concat([
    refined_stl_iq.head(520), records_iq.head(520)[['weekofyear']], training_targets_iq[['total_cases']]
], axis=1).dropna()
features_1_train_sj = pd.concat([
    refined_stl_sj.head(936), records_sj.head(936)[['weekofyear']], training_targets_sj[['total_cases']]
], axis=1).dropna()
features_1_test_iq = pd.concat([refined_stl_iq.tail(156), records_iq.tail(156)[['weekofyear']]], axis=1)
features_1_test_sj = pd.concat([refined_stl_sj.tail(260), records_sj.tail(260)[['weekofyear']]], axis=1)

In [ ]:
features_1_train_iq.to_csv('./generated/3-refined-' + str(version) + '-train-iq.csv')
features_1_train_sj.to_csv('./generated/3-refined-' + str(version) + '-train-sj.csv')
features_1_test_iq.to_csv('./generated/3-refined-' + str(version) + '-test-iq.csv')
features_1_test_sj.to_csv('./generated/3-refined-' + str(version) + '-test-sj.csv')

In [ ]:
selected_f_org_iq = feature_autoselect(corr_max=records_ml_corr_max_iq, 
                                       corr_argmax=records_ml_corr_argmax_iq, city='iq', max_lag=18, ntake=4)
selected_f_org_sj = feature_autoselect(corr_max=records_ml_corr_max_sj, 
                                       corr_argmax=records_ml_corr_argmax_sj, city='sj', max_lag=18, ntake=4)

In [ ]:
selected_f_org_iq

In [ ]:
selected_f_org_sj

In [ ]:
refined_org_iq = select_lag(records_iq, selected_f_org_iq)
refined_org_sj = select_lag(records_sj, selected_f_org_sj)

In [ ]:
features_0_train_iq = pd.concat([
    refined_org_iq.head(520), records_iq.head(520)[['weekofyear']], training_targets_iq[['total_cases']]
], axis=1).dropna()
features_0_train_sj = pd.concat([
    refined_org_sj.head(936), records_sj.head(936)[['weekofyear']], training_targets_sj[['total_cases']]
], axis=1).dropna()
features_0_test_iq = pd.concat([refined_org_iq.tail(156), records_iq.tail(156)[['weekofyear']]], axis=1)
features_0_test_sj = pd.concat([refined_org_sj.tail(260), records_sj.tail(260)[['weekofyear']]], axis=1)

In [ ]:
features_0_train_iq.to_csv('./generated/3-refined-o-' + str(version) + '-train-iq.csv')
features_0_train_sj.to_csv('./generated/3-refined-o-' + str(version) + '-train-sj.csv')
features_0_test_iq.to_csv('./generated/3-refined-o-' + str(version) + '-test-iq.csv')
features_0_test_sj.to_csv('./generated/3-refined-o-' + str(version) + '-test-sj.csv')